<a href="https://colab.research.google.com/github/carlos-alves-one/-ML-Zoomcamp-Week-3/blob/main/ML_Zoomcamp_Week_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Goldsmiths University of London
**Author....: Carlos Manuel de Oliveira Alves**<br>
**Student..: cdeol003**<br>
**Created..: 20/09/2022**

###3.1 Churn prediction project

In [15]:
# # Import libraries for the project
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [16]:
# Locate dataset for the project
data = 'https://github.com/alexeygrigorev/mlbookcamp-code/blob/fc43fe928f1b6d0a5143ce89110ddc135b1221e8/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'

In [18]:
# Download dataset and store it locally
!wget $data -O data-week-3.csv

--2022-09-20 10:53:41--  https://github.com/alexeygrigorev/mlbookcamp-code/blob/fc43fe928f1b6d0a5143ce89110ddc135b1221e8/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘data-week-3.csv’

data-week-3.csv         [  <=>               ]   2.62M  6.05MB/s    in 0.4s    

2022-09-20 10:53:42 (6.05 MB/s) - ‘data-week-3.csv’ saved [2742972]

